In [0]:
import pyspark.sql.functions as F
from  pyspark.sql.functions import col, struct, to_json
from pyspark.sql.types import StructField, StructType, StringType, MapType

In [0]:
jsonSchema = StructType([ StructField("title", StringType(), True), 
                         StructField("GlossEntry", StringType(), True) ])

In [0]:

inputPath = "dbfs:/FileStore/"

In [0]:
streamingInputDF = (
  spark
    .readStream 
    .option("multiline","true")                      
    .schema(jsonSchema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .json(inputPath)
)
display(streamingInputDF)

title,GlossEntry
null,null


In [0]:
display(streamingInputDF.selectExpr("title AS key", "to_json(struct(*)) AS value"))

key,value
null,{}


In [0]:
#write to topic
# bootstrap servers: is we need to keep our cluster server id
#username : we need to keep our cluster key username
#password: we need to keep our cluster key password
df= (streamingInputDF.selectExpr("title AS key", "to_json(struct(*)) AS value") \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "pkc-12576z.us-west2.gcp.confluent.cloud:9092") \
  .option("topic", "Basic_0") \
  .option("kafka.security.protocol","SASL_SSL") \
  .option("kafka.sasl.mechanism", "PLAIN") \
  .option("checkpointLocation", "dbfs:/FileStore/chekpoint/")
  .option("kafka.sasl.jaas.config", """kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="NJHYJFIAU2PR7ITR" password="Z+zNfpN/lo6I670vhizqXNgY8qemwsXc7fKmxLXb4PxkrUIUriDvJe9+E32vtFG1";""") \
  .start()
)

In [0]:
#Reading  to topic
# bootstrap servers: is we need to keep our cluster server id
#username : we need to keep our cluster key username
#password: we need to keep our cluster key password
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "pkc-12576z.us-west2.gcp.confluent.cloud:9092") \
      .option("subscribe", "Basic_0") \
      .option("startingOffsets", "earliest") \
      .option("kafka.security.protocol","SASL_SSL") \
      .option("kafka.sasl.mechanism", "PLAIN") \
      .option("kafka.sasl.jaas.config", """kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="NJHYJFIAU2PR7ITR" password="Z+zNfpN/lo6I670vhizqXNgY8qemwsXc7fKmxLXb4PxkrUIUriDvJe9+E32vtFG1";""") \
    .load()


In [0]:
json_schema = StructType(
    [   StructField("title", StringType(), True),
        StructField("GlossEntry", StringType(), True)
    ]
)

In [0]:
df3 = df.withColumn('value', F.from_json(F.col('value').cast('string'), json_schema)) \
        .select(F.col("value.title"), F.col("value.GlossEntry"))
display(df3)

display(df3)

title,GlossEntry
null,null
null,null
null,null
null,null
null,null
null,null
null,null
null,null
null,null
null,null
